In [ ]:
import os
os.environ["THEANO_FLAGS"] = "device=gpu0"
os.environ["CPATH"] = ""
import numpy as np

from collections import OrderedDict
from lasagnekit.misc.draw_net import draw_to_notebook, draw_to_file
from lasagnekit.misc.plot_weights import grid_plot
from lasagnekit.misc.dream import build_dreamer
from lasagnekit.easy import LightweightModel, BatchOptimizer
from lasagnekit.generative.capsule import Capsule
from lasagnekit.easy import BatchIterator, get_batch_slice


from lasagnekit.datasets.mnist import MNIST
from lasagnekit.datasets.fonts import Fonts
from lasagnekit.datasets.rescaled import Rescaled
from lasagnekit.datasets.dtd import DTD
from lasagnekit.datasets.cached import Cached

from lasagne import layers, updates
from IPython.display import SVG

import theano
import theano.tensor as T
from theano.tensor.shared_randomstreams import RandomStreams


srng = RandomStreams(seed=234)


from skimage.io import imread
from skimage.transform import resize

from lasagnekit.easy import get_stat

import matplotlib.pyplot as plt

from lasagne.layers import InputLayer, DenseLayer, NonlinearityLayer
from lasagne.layers import Conv2DLayer as ConvLayer
from lasagne.layers import Pool2DLayer as PoolLayer
from lasagne.nonlinearities import softmax, rectify
from lasagne.init import GlorotUniform, Constant
from sklearn.preprocessing import label_binarize
from sklearn.utils import shuffle
from sklearn.cross_validation import train_test_split

from utils import TransposedConv2DLayer, Conv2DDenseLayer

from caffezoo.googlenet import GoogleNet
%matplotlib inline

In [ ]:
googlenet = GoogleNet(input_size=(28, 28))
googlenet._load()

In [ ]:
seed = 1234
np.random.seed(seed)
rng = RandomStreams(seed)

In [ ]:
w, h = 28, 28 # Desired resolution, not necessarely the same than real_w and real_h, 
              # if necessary the images will be resize to fit w and h)
nb_colors = 3

In [ ]:
data = DTD(examples_filter=np.arange(200))
data.load()
data_rescaled = Cached(Rescaled(data, size=(w, h)))
data_rescaled.load()
X = data_rescaled.X
y = data_rescaled.y

In [ ]:
X = X.reshape((X.shape[0], nb_colors, w, h))

In [ ]:
for a in (googlenet._net.keys()):
    print(a)

In [ ]:
filter_size = 5
latent_size = 500
num_filters = 64

l_latent = googlenet._net["pool1/norm1"]



l_decoder1 = layers.Conv2DLayer(l_latent, 
                                filter_size=(3, 3),
                                pad='full',
                                num_filters=32)
l_decoder2 = layers.Conv2DLayer(l_decoder1, 
                                filter_size=(5, 5),
                                pad='full',
                                num_filters=32)
l_decoder3 = layers.Conv2DLayer(l_decoder2, 
                                filter_size=(7, 7),
                                pad='full',
                                num_filters=32)
l_decoder3 = layers.Conv2DLayer(l_decoder2, 
                                filter_size=(9, 9),
                                pad='full',
                                num_filters=32)
print(layers.get_output_shape(l_latent, (100, 3, 28, 28)))
#l_decoder_out = l_unconv
#x_to_z = LightweightModel([l_in], [l_latent])
#z_to_x = LightweightModel([l_latent], [l_decoder_out])
#model = Model()
#model.x_to_z = x_to_z
#model.z_to_x = z_to_x
#print(l_conv.W.get_value().shape)
#print(l_unconv.W.get_value().shape)